## Cleaning IMDB Data

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)


## Save each imdb file in the notebook

In [2]:
ratings = pd.read_csv("title.ratings.tsv", sep = '\t')
title_basic = pd.read_csv("title.basics.tsv", sep = '\t')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Edit id column
The title id column name is inconsistent across the databases. We need the names to be the same in order to merge them properly. Rename the title id column of each datacase so that they all are 'titleId'

In [3]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1823
1,tt0000002,6.0,232
2,tt0000003,6.5,1576
3,tt0000004,6.1,151
4,tt0000005,6.2,2400


## Merging data:
Merge each dataset by titleId
We chose left merge because at this point we want to keep all the information from the datasets. \
The newly combined dataset (movies) will have all the columns from the smaller datasets

In [4]:
movies = pd.merge(ratings,title_basic, on = 'tconst',how='outer')

## Filtering for Recent US TV shows
we only want English US tv series and we will have a seperate dataset for shows after 1997(done at very end). We filtered the columns of the dataset to match this description exactly We chose 1997 because this is the test netflix was release, so it will allow us to compare regular tv shows to those on streaming services.
The **release year** column was read as a string. so to make our >= condition work we changed the type to int <br>
To change the type to int we cannot have any nans in this column so we decided to remove rows with a Nan release year
## Why we chose to remove rows with NaN release_year
Instead of replacing these release data with other values such as 0 or researching the name of the series we decided to remove them. 
* Switching the data to 0 would make future anaysis confusing adn would preserve the meaning of the column.
* When looking through these NaN release year, a lot of the rows were not conventual tv series. They were non-english, documentaries, animes or musicals that were certainly not made after 1997. ex: hamlet, valhalla club documentary. 
* These rows were also likely to have other columns missing information such as the rating column and other important categories, so we decided by removing them, we were not losing much information


In [5]:
movies= movies[movies['startYear'] != '\\N']

In [6]:
movies= movies[movies['startYear'].notna()]

In [7]:
movies['startYear'] = movies['startYear'].astype(int)

**Filters for tv series only**

In [8]:
movies = movies[movies['titleType'] == 'movie']

In [9]:
movies['titleType'].unique()

array(['movie'], dtype=object)

**Filters for english only shows**

In [10]:
movies = movies[movies['originalTitle'] == movies['primaryTitle']]

## Removing more Nans
We decided it would be best to remove rows with a nan average rating score since this is how we determine if a tv series is successful or not. Not having this column would prevent us from doing regressions, correlations etc so its best to remove it

In [11]:
movies = movies[movies['averageRating'].notna()]

In [12]:
movies.head()

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
340,tt0000502,4.5,14.0,movie,Bohemios,Bohemios,0,1905,\N,100,\N
374,tt0000574,6.1,697.0,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
397,tt0000615,4.5,23.0,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama
435,tt0000675,4.9,19.0,movie,Don Quijote,Don Quijote,0,1908,\N,\N,Drama
438,tt0000679,5.3,65.0,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,\N,120,"Adventure,Fantasy"


## Standardize column names

In [13]:
movies = movies.rename(columns={'averageRating': 'rating','genres':'genre', 'primaryTitle':'title' })

## dropping columns
Next we will drop the columns we are not usings for our analysis to make the dataset easier to read

In [14]:
movies =movies.drop(['titleType','tconst'], axis = 1)

In [15]:
movies.to_csv("movies.csv", index = False)